# ODE Integrators III: Advanced Topics

## Adaptive Stepsize Control

When we integrate ordinary differential equations (ODEs), the choice of time step $\Delta t$ can influence the accuracy and efficiency of our numerical solutions.
This issue becomes especially important for systems that exhibit rapid changes in time and/or chaotic behavior.
In such cases, tiny differences in initial conditions or time step selection can lead to wildly different outcomes.
This highlights why we must continually monitor and control numerical errors.

**Adaptive step size control** provides a systematic way to handle these challenges.
Instead of using a fixed $\Delta t$, adaptive methods dynamically change the time step based on error estimates at each step.
The critical idea behind adaptive step size control is to balance two competing goals:
* **Accuracy:** Keep truncation (local) errors within a user-specified tolerance.
* **Efficiency:** Avoid making the time step too small, which would waste computational resources.

In practice, we estimate the local error of a proposed step and compare it to a tolerance that combines both absolute and relative components.
If the error is too large, we reduce the step size.
If it is safely below the tolerance, we may increase it.
This approach automatically "zooms in" when the ODE solution changes rapidly and "zooms out" when the solution is smoother.

### Error Estimate

To illustrate one of the simplest ways to estimate errors, consider advancing the solution from $t$ to $t + 2\Delta t$.
There are two ways to do this:
1. Step the ODE system with a single step $\Delta t' = 2\Delta t$.
2. Step the ODE system with two consecutive steps, each step is $\Delta t$.

For a fourth-order method (e.g., classic Runge–Kutta 4), the local truncation errors differ between these two approaches.
Symbolically, the resulting solutions (ignoring higher-order terms) look like:
\begin{align}
\text{One step:}  \quad x(t + 2\Delta t) &= x_1 + (2\Delta t)^5 \phi + \mathcal{O}(\Delta t^6) + \dots \\
\text{Two steps:} \quad x(t + 2\Delta t) &= x_2 +  2\Delta t^5  \phi + \mathcal{O}(\Delta t^6) + \dots
\end{align}
where $\phi$ is some function of $t$ and $x(t)$.
We focus on just the leading $\Delta t^5$ terms and define
\begin{align}
\Delta = x_2 - x_1,
\end{align}
to give an approximate measure of the local truncation error.
If $\Delta$ is "too large," we should reduce $\Delta t$.
If $\Delta$ is "too small," we might be over-resolving the solution and can safely increase $\Delta t$.

### Richardson Extrapolation

An interesting by-product of step doubling is Richardson extrapolation, which can yield a higher-order estimate:
\begin{align}
x(t + 2\Delta t) = x_2 + \frac{\Delta}{15} + \mathcal{O}(\Delta t^6)
\end{align}
While this estimate is more accurate (fifth-order), we have no direct way to estimate or control its truncation error at that higher order.
Therefore, it is not as convenient for adaptive step size selection as one might initially hope.

### Embedded Runge-Kutta Formulas: The Dormand–Prince Method

Step doubling is a simple adaptive step method.
However, it is now mostly replaced by a more efficient stepsize adjustment algorithm based on embedded Runge-Kutta formulas, originally invented by Merson and popularized in a method of Fehlberg.

An interesting fact about Runge-Kutta formulas is that for orders $M$ higher than four, more than $M$ function evaluations are required.
This accounts for the popularity of the classical fourth-order method: It seems to give the most bang for the buck. However, Fehlberg discovered a fifth-order method with six function evaluations where another combination of the six functions gives a fourth-order method.

The difference between the two estimates of $y(x + \Delta t)$ can then be used as an estimate of the truncation error to adjust the stepsize.
Since Fehlberg's original formula, many other embedded Runge-Kutta formulas have been found.

The Dormand–Prince method is a 5th-order Runge-Kutta method with an embedded 4th-order solution.
This pairing enables accurate error estimation, which is essential for adaptive step size control.
The DP method is particularly favored for its robustness and efficiency, making it a staple in many numerical computing libraries, including MATLAB's ode45.

The Dormand–Prince method is characterized by its specific set of coefficients, which dictate how intermediate slopes are calculated and combined to produce the final solution estimates.
These coefficients are meticulously chosen to minimize error and optimize computational performance.

The general form of a fifth-order Runge-Kutta formula is
\begin{align}
k_1 &= \Delta t f(x_n, t_n)\\
k_2 &= \Delta t f(x_n + a_{21}k_1, t_n + c_2 \Delta t)\\
\cdots\\
k_6 &= \Delta t f(x_n + a_{61}k_1 + \cdots + a_{65}k_5, t_n + c_6 \Delta t)\\
x_{n+1} &= x_n + b_1 k_1 + b_2 k_2 + \cdots + b_6 k_6 + \mathcal{O}(\Delta t^6)
\end{align}

The Dormand–Prince method employs a set of coefficients $a$, $b$, and $c$ to compute intermediate stages and combine them into higher and lower-order solutions.
Below are the coefficients for the DP method:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
a = [
    [],
    [1/5],
    [3/40, 9/40],
    [44/45, -56/15, 32/9],
    [19372/6561, -25360/2187, 64448/6561, -212/729],
    [9017/3168, -355/33, 46732/5247, 49/176, -5103/18656],
    [35/384, 0, 500/1113, 125/192, -2187/6784, 11/84],
]

b_high = [35/384, 0, 500/1113, 125/192, -2187/6784, 11/84, 0] # Fifth-order accurate solution estimate
b_low  = [5179/57600, 0, 7571/16695, 393/640, -92097/339200, 187/2100, 1/40] # Fourth-order accurate solution estimate

c = [0, 1/5, 3/10, 4/5, 8/9, 1, 1]

A DP45 step is therefore

In [ ]:
def DP45_step(f, x, t, dt):
        # Compute intermediate k1 to k7
    k1 = np.array(f(*x))
    k2 = np.array(f(*(x + dt*(a[1][0]*k1))))
    k3 = np.array(f(*(x + dt*(a[2][0]*k1 + a[2][1]*k2))))
    k4 = np.array(f(*(x + dt*(a[3][0]*k1 + a[3][1]*k2 + a[3][2]*k3))))
    k5 = np.array(f(*(x + dt*(a[4][0]*k1 + a[4][1]*k2 + a[4][2]*k3 + a[4][3]*k4))))
    k6 = np.array(f(*(x + dt*(a[5][0]*k1 + a[5][1]*k2 + a[5][2]*k3 + a[5][3]*k4 + a[5][4]*k5))))
    k7 = np.array(f(*(x + dt*(a[6][0]*k1 + a[6][1]*k2 + a[6][2]*k3 + a[6][3]*k4 + a[6][4]*k5 + a[6][5]*k6))))

    ks = [k1, k2, k3, k4, k5, k6, k7]

    # Compute high and low order estimates
    x_high = x + dt * np.dot(b_high, ks)
    x_low  = x + dt * np.dot(b_low,  ks)

    return x_high, x_low, ks

### Proportional-Integral Step Size Control

Once we have an embedded Runge-Kutta method like Dormand–Prince in place, the next step is to implement a mechanism to adjust the step size based on the estimated local error.
The PI (Proportional-Integral) controller is a widely-used strategy for this purpose, combining proportional and integral components to achieve stable and efficient step size adjustments.

The PI controller adjusts the step size $\Delta t$ based on the current error $E$ and past errors.
The objective is to maintain the error within specified tolerances while preventing drastic changes in step size that could lead to instability or inefficiency.

The general formula for updating the step size is:
\begin{align}
h_{\text{new}} = h \cdot \min\left(\text{fac}{\text{max}}, \max\left(\text{fac}{\text{min}}, \text{fac} \cdot \left(\frac{\text{tol}}{E}\right)^{\alpha}\right)\right)
\end{align}
where $\text{fac}$ is a scaling factor (typically around 0.9) to provide a safety margin;
$\text{fac}{\text{min}}$ and $\text{fac}{\text{max}}$ set the minimum and maximum allowable step size multipliers to prevent excessive changes; and
$\alpha$ is an exponent that determines the responsiveness of the step size adjustment.

In [ ]:
def dt_update(dt, error, tol, fac=0.9, fac_min=0.1, fac_max=4.0, alpha=0.2):
    if error == 0:
        s = fac_max
    else:
        s = fac * (tol / error) ** alpha
    s = min(fac_max, max(fac_min, s))
    dt_new = dt * s
    return dt_new

Combining the single embedded step and the step controller, we obtain the DP45 algorithm:

In [ ]:
def DP45(f, x, t, T, dt, atol, rtol):

    Ts = [t]
    Xs = [np.array(x)]

    while t < T:
        if t + dt > T:
            dt = T - t  # Adjust step size to end exactly at tf

        # Perform a single Dormand–Prince step
        x_high, x_low, _ = DP45_step(f, x, t, dt)

        # Compute the error estimate
        error = np.linalg.norm(x_high - x_low, ord=np.inf)

        # Compute the tolerance
        tol = atol + rtol * np.linalg.norm(x_high, ord=np.inf)

        # Check if the step is acceptable
        if error <= tol:
            # Accept the step
            t += dt
            x = x_high
            Ts.append(t)
            Xs.append(x)

        # Compute the new step size
        dt = dt_update(dt, error, tol)

    return np.array(Ts), np.array(Xs)

We can now apply it to the ODEs.

Applying to the simple harmonic oscillator, we may specifically ask for the accuracy:

In [ ]:
def f_sh(theta, omega):
    return omega, -theta

In [ ]:
def error_DP45(tol):
    T, X = DP45(f_sh, (0, 0.01), 0, 10, 0.1, tol, tol)
    Theta  = X[:,0]
    Thetap = 0.01 * np.sin(T)
    return np.max(abs(Theta - Thetap))

N     = np.array([64, 128, 256, 512, 1024])
EDP45 = np.array([error_DP45(tol) for tol in 2.0**(-22-4*np.arange(5))])

plt.loglog(N, 1/N**4,      label='1/N^4')
plt.loglog(N, EDP45, 'o--', label='RK38')
plt.xlabel('N')
plt.ylabel(r'$\text{err} = max|x_\text{numeric} - x|$')
plt.legend()

For non-linear problems, we can compare different accuracies:

In [ ]:
def f_dp(th1, th2, p1, p2):
    m  = 1
    l  = 1
    g  = 1

    u1 = m * l * l
    u2 = g / l
    f  = 6 / (u1 * (16 - 9 * np.cos(th1 - th2)**2))

    dth1 = f * (2 * p1 - 3 * np.cos(th1 - th2) * p2)
    dth2 = f * (8 * p2 - 3 * np.cos(th1 - th2) * p1)

    dp1  = - 0.5 * u1 * (  dth1 * dth2 * np.sin(th1 - th2) + 3 * u2 * np.sin(th1))
    dp2  = - 0.5 * u1 * (- dth1 * dth2 * np.sin(th1 - th2) +     u2 * np.sin(th2))

    return dth1, dth2, dp1, dp2

In [ ]:
for i, atol in enumerate([1e-3,1e-6,1e-9,1e-12]):
    T, X = DP45(f_dp, (np.pi/2, np.pi/2, 0.0, 0.0), 0, 10, 0.1, atol, 0)
    plt.plot(T[::10], X[::10,0], '.-',  color=f'C{i}', label=f'atol={atol}')
    plt.plot(T[::10], X[::10,1], '.--', color=f'C{i}')
plt.legend()

Up to this point, we have explored the advanced concepts of Adaptive Step Size Control in Runge-Kutta methods, focusing on the Dormand–Prince (DP) method.
We began by understanding the significance of embedded Runge-Kutta formulas, which enable simultaneous computation of solutions of different orders for effective error estimation.
This foundation allowed us to implement a PI controller that dynamically adjusts the integration step size based on local error estimates, ensuring that the numerical solution remains within desired accuracy bounds while optimizing computational efficiency.